In [24]:
from django_for_jupyter import init_django
init_django()

In [2]:
from home.models import Book_Topic, Book
import pandas as pd
# from django_pandas.io import read_frame
books = Book.objects.all()
bookTopic = Book_Topic.objects.prefetch_related('topic_id')
check = True
dics = []
for book in books:
    content = book.book_title
    topics = bookTopic.filter(book_id = book.book_id)
    for topic in topics:
        content+=' '+topic.topic_id.topic_name
    dic = {'book_id': book.book_id,
            'content': content
           }
    dics.append(dic) 
book_df = pd.DataFrame(dics)
print(book_df)

    book_id                                            content
0      3032  Cisco The complete reference Computer network ...
1      3001  CDMA capacity and quality optimization code di...
2      3002  Image databases : Search and retrieval of digi...
3      3003  Practical handbook on image processing for sci...
4      3004  Analytics, data science, & artificial intellig...
..      ...                                                ...
71     3072  Giáo trình lập trình .Net Computer programming...
72     3073  Giáo trình đảm bảo chất lượng phần mềm Softwar...
73     3074  Data engineering Fuzzy mathematics in systems ...
74     3075  Digital image processing Image processing Digi...
75     3076  Digital image processing using MATLAB Image pr...

[76 rows x 2 columns]


In [3]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

In [4]:

book_tfidf = TfidfVectorizer(stop_words='english')
book_df['content'] = book_df['content'].fillna('')
book_content_matrix = book_tfidf.fit_transform(book_df['content'])
book_content_matrix.shape

(76, 268)

In [5]:
cosine_similarity = linear_kernel(book_content_matrix, book_content_matrix)

In [10]:
choice = 68
similarity_scores = list(enumerate(cosine_similarity[choice-1]))
similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
similarity_scores = similarity_scores[1:6]

# Get the similar books index
books_index = [i[0] for i in similarity_scores]

# printing the top 5 most similar books using integer-location based indexing (iloc)
print(f'The choice is: {books[choice-1].book_title}')
for i in books_index:
    print(f'{books[i].book_id} - {books[i].book_title} ')


The choice is: Giáo trình ngôn ngữ mô hình hóa UML
3049 - Giáo trình lập trình căn bản với ngôn ngữ C 
3040 - Software engineering : A practitioner's approach 
3071 - Giáo trình kiến trúc và thiết kế phần mềm 
3042 - Object-oriented software engineering : using UML, patterns, and Java 
3052 - Giáo trình bảo trì phần mềm 


In [32]:
from home.models import Book, Rating
from django.contrib.auth.models import User
# method insert rating cho mot cuon sach
userid = 1
bookid = 3001
point = 4
rate = Rating.objects.filter(user_id = userid, book_id = bookid).first()
if(not rate):
    print('user has not rated this book')
    # insert rating
    rate1 = Rating(user_id = userid, book_id = bookid, rating = point)
    rate1.save()
else:
    # update rating
    rate.rating = point
    rate.save()
    print("update rating")


update rating


In [1]:
from django_for_jupyter import init_django
init_django()

In [9]:
from home.models import Book_Topic, Topic, Book, Rating
from django.db.models import Avg
book_id = 3001
def averRating(book_id):
    Rating.objects.filter(book_id=book_id).aggregate(ave=Avg('rating'))['ave']
print(averRating(book_id))

4.0
